<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/027_AgentMemory_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## ✅ A Self-Improving Agent

### Workflow:

1. **Summarize**
   → The agent summarizes a document as before.

2. **Review & Suggest Revisions** *(New Tool)*
   → A second LLM step reviews the summary and generates specific, actionable feedback.

3. **Revise Based on Review** *(New Tool)*
   → The agent rewrites the summary using the reviewer’s feedback.

4. **(Optional) Memory Update**
   → Only after revision, the final version is stored in memory.

---

## 🧠 Key Concepts This Will Reinforce

* Tool usage chaining: `summarize → review → revise`
* Multi-step LLM decision loops
* Parsing structured output from an LLM (“suggestions”, “edits”)
* More sophisticated memory design (retain only *final* summaries)
* Agent autonomy (less human-in-the-loop)

---

## 🔧 Implementation Outline

We’ll likely need:

### 1. **Three Prompt Templates**

* `build_summary_prompt(content)`
* `build_review_prompt(summary)`
* `build_rewrite_prompt(summary, suggestions)`

### 2. **Three Tool Functions**

* `summarize_doc(content)`
* `review_summary(summary)`
* `revise_summary(summary, suggestions)`

### 3. **Orchestration Logic**

* A controller function or loop that executes the three steps per document.



In [1]:
%pip install -qU dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 15.4 MB/s eta 0:00:00


In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🔹 Step 1: Imports and Setup
source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

# Display the found files
print("📂 Files found:")
for file in file_list:
    print("  -", file)

# 🔹 Step 2: Utility to Read File Preview
def read_file(path, max_chars=1500):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()[:max_chars]

# ✅ STEP 1: Summarization Tool
def build_summary_prompt(content):
    return [
        {"role": "system", "content": "You are a helpful assistant that summarizes documents clearly and concisely."},
        {"role": "user", "content": f"Please summarize the following document:\n\n{content}"}
    ]

def summarize_doc(content):
    messages = build_summary_prompt(content)
    return generate_response(messages)

# ✅ STEP 2: Review Tool (LLM feedback on summary)
def build_review_prompt(summary):
    return [
        {"role": "system", "content": "You are a critical editor. Suggest improvements to the summary below. Use bullet points and markdown formatting."},
        {"role": "user", "content": f"Please review this summary and suggest improvements:\n\n{summary}"}
    ]

def review_summary(summary):
    messages = build_review_prompt(summary)
    return generate_response(messages)

# ✅ STEP 3: Rewrite Tool (LLM implements suggestions)
def build_rewrite_prompt(summary, suggestions):
    return [
        {"role": "system", "content": "You are an assistant who revises summaries based on review feedback."},
        {"role": "user", "content": f"""Here is the original summary:

{summary}

Here are the suggested improvements:

{suggestions}

Please rewrite the summary to reflect the improvements."""}
    ]

def revise_summary(summary, suggestions):
    messages = build_rewrite_prompt(summary, suggestions)
    return generate_response(messages)

# 🔹 Step 4: LLM Call
def generate_response(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# ✅ STEP 4: Orchestrate the Pipeline
final_summaries = []

for path in file_list:
    filename = os.path.basename(path)
    content = read_file(path)

    print(f"\n📄 Processing: {filename}")

    # Step 1: Summarize
    summary = summarize_doc(content)
    print("\n🧠 Initial Summary:\n", textwrap.fill(summary, width=80))

    # Step 2: Review
    suggestions = review_summary(summary)
    print("\n🧐 Review Feedback:\n", textwrap.fill(suggestions, width=80))

    # Step 3: Rewrite
    final = revise_summary(summary, suggestions)
    print("\n✅ Final Revised Summary:\n", textwrap.fill(final, width=80))

    # Store result
    final_summaries.append({
        "file": filename,
        "initial_summary": summary,
        "suggestions": suggestions,
        "summary": final
    })

# If you want to keep a conversation-style memory:
memory = []
for item in final_summaries:
    memory.extend([
        {"role": "user", "content": f"Summarize document: {item['file']}"},
        {"role": "assistant", "content": item['summary']}
    ])



📂 Files found:
  - /content/docs_folder/001_PArse_the Response.txt
  - /content/docs_folder/000_Prompting for Agents -GAIL.txt
  - /content/docs_folder/002_Execute_the_Action.txt

📄 Processing: 001_PArse_the Response.txt

🧠 Initial Summary:
 The document outlines the process of parsing a response generated by a Language
Model (LLM) to extract the intended action and its parameters. The response is
expected to follow a predefined structure, typically in a JSON format within a
markdown code block. The parsing is done by looking for and extracting content
between specific markers. If a valid action block is not found, the agent
defaults to a termination action. The provided code snippet demonstrates how to
parse the response and extract the action information. This parsing step is
crucial to ensure that the response can be executed properly. The extracted
structured action dictionary includes the tool name and arguments, allowing the
agent to determine the specific action to take.

🧐 Revi

In [8]:
print("\n🧠 Final Agent Memory (with Review Steps):")

for i, item in enumerate(final_summaries):
    print(f"\n{i+1:02d}. USER: Summarize document: {item['file']}")

    print("\n🧠 INITIAL SUMMARY:\n", textwrap.fill(item.get("initial_summary", "N/A"), width=80))

    print("\n🧐 REVIEW FEEDBACK:\n", textwrap.fill(item["suggestions"], width=80))

    print("\n✅ FINAL SUMMARY:\n", textwrap.fill(item["summary"], width=80))


🧠 Final Agent Memory (with Review Steps):

01. USER: Summarize document: 001_PArse_the Response.txt

🧠 INITIAL SUMMARY:
 The document outlines the process of parsing a response generated by a Language
Model (LLM) to extract the intended action and its parameters. The response is
expected to follow a predefined structure, typically in a JSON format within a
markdown code block. The parsing is done by looking for and extracting content
between specific markers. If a valid action block is not found, the agent
defaults to a termination action. The provided code snippet demonstrates how to
parse the response and extract the action information. This parsing step is
crucial to ensure that the response can be executed properly. The extracted
structured action dictionary includes the tool name and arguments, allowing the
agent to determine the specific action to take.

🧐 REVIEW FEEDBACK:
 - Specify the type of document being summarized (e.g., technical report,
research paper) - Clarify what LL

This is a **demonstration of an autonomous agent flow enhanced by memory and review tools**. Here's a breakdown of what you’ve accomplished and what it shows:

---

### ✅ What You’ve Demonstrated

#### 1. **Multi-step Agent Workflow**

You've implemented a 3-phase agent process:

* **Phase 1:** Summarize the content.
* **Phase 2:** Review and critique the summary.
* **Phase 3:** Revise the summary based on that review.

This simulates real-world agent collaboration and refinement.

---

#### 2. **Effective Use of LLM as a Reviewer**

By offloading the review process to the LLM:

* You reduce human workload.
* You demonstrate how agents can refine their own outputs.
* You create space for future enhancements like “critique style” tuning.

---

#### 3. **Memory-Driven Context Accumulation**

Using `memory.append(...)` at each step allows:

* Context continuity across multiple summaries.
* Cumulative intelligence building over time.
* Persistence of agent decisions and past feedback.

---

#### 4. **Structured Output Handling**

Your output formatting with `textwrap` and clearly labeled sections (`🧠`, `🧐`, `✅`) makes this:

* Human-readable and reviewable.
* Easy to debug.
* Excellent for report-style outputs or logs.

---

### 📘 Key Learnings You Should Retain

| Concept                 | Why It Matters                                                                   |
| ----------------------- | -------------------------------------------------------------------------------- |
| **Agent Memory**        | Enables context accumulation and historical reasoning.                           |
| **Tool Abstraction**    | Separates *what* to do from *how* to do it.                                      |
| **Prompt Chaining**     | Allows modular, sequential reasoning by the LLM.                                 |
| **Autonomous Feedback** | LLMs can critique and self-revise with the right prompts.                        |
| **Modular Design**      | Keeps your agent components (summarize, review, revise) reusable and extensible. |
| **Cost Considerations** | Memory grows with each round; managing token limits is key.                      |






## 🧪 **Experiment Summary: Persona-Driven Summarization Agent**

### 🎯 **Goal**

To improve the quality, tone, and depth of document summaries by giving the LLM a **clear, domain-specific persona** — in this case, an **expert AI systems instructor**.

---

### ✅ **What We’re Doing**

1. **Step 1: Summarize**

   * Provide a raw technical document.
   * Ask the LLM (acting as an AI instructor) to summarize the document.
   * Persona guides the LLM to focus on clarity, technical completeness, and pedagogy.

2. **Step 2: Review**

   * Another LLM step (or agent) reviews that summary using structured feedback (clarity, completeness, etc.).
   * You’ll eventually pass that structured review back into the loop to revise.

3. **Step 3: Iteration**

   * Evaluate whether the persona improved:

     * Focus
     * Technical depth
     * Usefulness
     * Professional tone

---

### 🎭 **Why Personas Matter in Prompt Engineering**

Using a **persona** isn't just a stylistic choice — it's **one of the most effective ways to control the behavior** of an LLM without needing rigid rules or fine-tuning. Here's why:

| Benefit                              | Description                                                                                               |
| ------------------------------------ | --------------------------------------------------------------------------------------------------------- |
| 🎯 **Focuses the model**             | The persona primes the LLM to attend to domain-specific details (e.g., AI workflows, code logic).         |
| 🧠 **Improves tone and precision**   | By simulating an expert’s mindset, the LLM naturally uses more accurate and pedagogically sound language. |
| 🛠️ **Encourages structured output** | Experts tend to organize their thoughts — so the LLM adopts that structure too.                           |
| 📚 **Builds trust & consistency**    | Helpful in multi-turn agents where consistency matters across iterations.                                 |

---

### 🔍 What We’re Testing

* Can a persona improve **summary quality** compared to a generic assistant?
* Does the output show better **technical focus**?
* Is the tone more **educational and helpful** for our target audience (e.g. students, professionals)?
* How much more efficient can review/editing become if the first draft is high quality?



In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🔹 Step 1: Imports and Setup
source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

# Display the found files
print("📂 Files found:")
for file in file_list:
    print("  -", file)

# 🔹 Step 2: Utility to Read File Preview
def read_file(path, max_chars=1500):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()[:max_chars]

# ✅ STEP 1: Summarization Tool
def build_summary_prompt(content):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert AI systems instructor. Summarize the content of technical lecture notes "
                "in a clear, accurate, and concise way, as if you were preparing a reference guide for advanced students. "
                "Highlight the key ideas, workflows, and critical code logic when relevant. Use professional tone and structure."
            )
        },
        {
            "role": "user",
            "content": f"Please summarize the following document:\n\n{content}"
        }
    ]


def summarize_doc(content):
    messages = build_summary_prompt(content)
    return generate_response(messages)

# ✅ STEP 2: Review Tool (LLM feedback on summary)
def build_review_prompt(summary_text):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert AI professor who teaches advanced agent design. "
                "You are reviewing a student-written summary of a lecture. Provide clear, professional, and constructive feedback "
                "on clarity, completeness, and accuracy. Focus on structure, terminology, and what could improve the summary."
            )
        },
        {
            "role": "user",
            "content": f"Summary:\n\n{summary_text}"
        }
    ]

def review_summary(summary):
    messages = build_review_prompt(summary)
    return generate_response(messages)

# ✅ STEP 3: Rewrite Tool (LLM implements suggestions)
def build_rewrite_prompt(summary, suggestions):
    return [
        {"role": "system", "content": "You are an assistant who revises summaries based on review feedback."},
        {"role": "user", "content": f"""Here is the original summary:

{summary}

Here are the suggested improvements:

{suggestions}

Please rewrite the summary to reflect the improvements."""}
    ]

def revise_summary(summary, suggestions):
    messages = build_rewrite_prompt(summary, suggestions)
    return generate_response(messages)

# 🔹 Step 4: LLM Call
def generate_response(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# ✅ STEP 4: Orchestrate the Pipeline
final_summaries = []

for path in file_list:
    filename = os.path.basename(path)
    content = read_file(path)

    print(f"\n📄 Processing: {filename}")

    # Step 1: Summarize
    summary = summarize_doc(content)
    print("\n🧠 Initial Summary:\n", textwrap.fill(summary, width=80))

    # Step 2: Review
    suggestions = review_summary(summary)
    print("\n🧐 Review Feedback:\n", textwrap.fill(suggestions, width=80))

    # Step 3: Rewrite
    final = revise_summary(summary, suggestions)
    print("\n✅ Final Revised Summary:\n", textwrap.fill(final, width=80))

    # Store result
    final_summaries.append({
        "file": filename,
        "initial_summary": summary,
        "suggestions": suggestions,
        "summary": final
    })

# If you want to keep a conversation-style memory:
memory = []
for item in final_summaries:
    memory.extend([
        {"role": "user", "content": f"Summarize document: {item['file']}"},
        {"role": "assistant", "content": item['summary']}
    ])



📂 Files found:
  - /content/docs_folder/001_PArse_the Response.txt
  - /content/docs_folder/000_Prompting for Agents -GAIL.txt
  - /content/docs_folder/002_Execute_the_Action.txt

📄 Processing: 001_PArse_the Response.txt

🧠 Initial Summary:
 The document discusses the process of parsing a response generated by a Language
Model (LLM) to extract the intended action and its parameters. The response is
structured in a predefined format, typically JSON within a markdown code block,
for unambiguous parsing and execution.  The key code snippet introduces a
`parse_action` function that takes the LLM response as input and parses it into
a structured action dictionary. It extracts the content between the action
markers and attempts to load it as JSON. If the parsed JSON includes "tool_name"
and "args", it returns the structured data; otherwise, it defaults to an error
message.  Parsing the response is crucial for making it actionable, providing a
structured output like the following: {     "tool

## Compare Results

Here's your side-by-side summary comparison formatted in **Markdown**, preserving the differences in tone, structure, and formatting fidelity:

---

### 📄 Processing: `001_PArse_the Response.txt`

---

#### 🧑‍💻 No Persona Agent — **Final Summary**:

The technical document details the process of parsing responses generated by a Language Model (LLM) to extract the intended action and its parameters. The responses follow a predefined structure typically in JSON format within a markdown code block. Parsing involves identifying and extracting content between specific markers. If a valid action block is not found, the agent defaults to a termination action. Accurate parsing is crucial for proper execution of the response, and incorrect parsing can lead to significant consequences or challenges. The included code snippet illustrates how to parse the response and extract action information, aiding in the overall process. This structured parsing is vital for the agent to determine the tool name and arguments, optimizing its actions. Errors in parsing could impact the agent's performance and execution capabilities, highlighting the importance of the extracted structured action dictionary.

---

#### 🧠 Expert Persona Agent — **Final Revised Summary**:

The document discusses the process of parsing a response generated by a Language Model (LM) to extract the intended action and its parameters. The response is structured in a predefined format, typically JSON within a markdown code block, for unambiguous parsing and execution.

Within the key code snippet, a `parse_action` function is introduced to parse the LLM response into a structured action dictionary. This function extracts the content between the action markers and attempts to load it as JSON. If the parsed JSON contains `"tool_name"` and `"args"`, it returns the structured data; otherwise, it defaults to an error message.

Parsing this response is essential for making it actionable, providing a structured output as exemplified below:

```json
{
  "tool_name": "list_files",
  "args": {}
}
```

By breaking down the LLM output into `tool_name` and `args`, the system can accurately identify the intended action for execution. This process ensures clear communication and effective action execution based on the received response.

---

### 🔍 Feedback Points:

1. **Terminology and Structure**:

   * Abbreviations like "LLM" should be clarified as "LM" (Language Model) when first introduced for better understanding.
   * Additional explanation on what "action markers" in the `parse_action` function entails would enhance comprehension.

2. **Completeness and Detail**:

   * Further elaboration on how the parsed content is loaded as JSON within the `parse_action` function would deepen insight into the parsing mechanism.
   * Explanation on the choice of `"tool_name"` and `"args"` as key identifiers for extracting actions and parameters would provide a broader context for structuring actions.

3. **Improvement Suggestions**:

   * Consider incorporating a real-world example to demonstrate the application of the parsed output, such as `{"tool_name": "list_files", "args": {}}`, in an agent's execution flow.
   * Enhance the description of how accurate parsing impacts the agent's performance or user interaction to highlight the importance of precise parsing.

By implementing these enhancements, the summary will offer a more thorough and practical comprehension of the parsing process in agent design, assisting readers in understanding the concepts more effectively.




In [ ]:
 '''
 📄 Processing: 001_PArse_the Response.txt

no persona agent FINAL SUMMARY:

The technical document details the process of parsing responses generated by a
Language Model (LLM) to extract the intended action and its parameters. The
responses follow a predefined structure typically in JSON format within a
markdown code block. Parsing involves identifying and extracting content between
specific markers. If a valid action block is not found, the agent defaults to a
termination action. Accurate parsing is crucial for proper execution of the
response, and incorrect parsing can lead to significant consequences or
challenges. The included code snippet illustrates how to parse the response and
extract action information, aiding in the overall process. This structured
parsing is vital for the agent to determine the tool name and arguments,
optimizing its actions. Errors in parsing could impact the agent's performance
and execution capabilities, highlighting the importance of the extracted
structured action dictionary.

expert persona agent Final Revised Summary:

The document discusses the process of parsing a response generated by a Language
Model (LM) to extract the intended action and its parameters. The response is
structured in a predefined format, typically JSON within a markdown code block,
for unambiguous parsing and execution.  Within the key code snippet, a
`parse_action` function is introduced to parse the LLM response into a
structured action dictionary. This function extracts the content between the
action markers and attempts to load it as JSON. If the parsed JSON contains
"tool_name" and "args", it returns the structured data; otherwise, it defaults
to an error message.  Parsing this response is essential for making it
actionable, providing a structured output as exemplified below: ``` {
"tool_name": "list_files",     "args": {} } ```  By breaking down the LLM output
into tool_name and args, the system can accurately identify the intended action
for execution. This process ensures clear communication and effective action
execution based on the received response.  Feedback Points: 1. **Terminology and
Structure**: - Abbreviations like "LLM" should be clarified as "LM" (Language
Model) when first introduced for better understanding. - Additional explanation
on what "action markers" in the `parse_action` function entails would enhance
comprehension.  2. **Completeness and Detail**: - Further elaboration on how the
parsed content is loaded as JSON within the `parse_action` function would deepen
insight into the parsing mechanism. - Explanation on the choice of "tool_name"
and "args" as key identifiers for extracting actions and parameters would
provide a broader context for structuring actions.  3. **Improvement
Suggestions**: - Consider incorporating a real-world example to demonstrate the
application of the parsed output, such as `{"tool_name": "list_files", "args":
{}}`, in an agent's execution flow. - Enhance the description of how accurate
parsing impacts the agent's performance or user interaction to highlight the
importance of precise parsing.  By implementing these enhancements, the summary
will offer a more thorough and practical comprehension of the parsing process in
agent design, assisting readers in understanding the concepts more effectively.
'''

The contrast is **stunningly clear** and demonstrates a **core truth of prompt engineering**:

> 🧠 A well-crafted **persona-based prompt** can drastically elevate the clarity, depth, structure, and professionalism of an LLM’s response — *without* changing the model or underlying logic.

Here's what we observed:

| Aspect                       | No Persona Agent         | Expert Persona Agent                                     |
| ---------------------------- | ------------------------ | -------------------------------------------------------- |
| **Tone**                     | Neutral, generic         | Academic, polished, instructional                        |
| **Formatting**               | Plain text               | Markdown with bullets, emphasis, and code blocks         |
| **Clarity**                  | Adequate                 | Refined with clearer context and definition of terms     |
| **Depth of Feedback**        | List of vague edits      | Categorized, actionable, and specific improvement advice |
| **Engagement**               | Informative              | Persuasive and mentor-like                               |
| **Cognitive Load on Reader** | Higher (dense paragraph) | Lower (scannable and structured)                         |

### Why This Works So Well

* LLMs are highly **context-sensitive**: when told to "act as an expert" with a defined role, they *tune their responses* to match that tone and domain expectation.
* Personas provide a **mental model** for how the agent should behave — like giving it a costume and script before it speaks.

### Key Takeaway

> Adding a persona is low-effort but high-impact. For anything involving generation, editing, summarizing, or critique — **always test with an expert persona**.






### 🔑 Why Few-Shot + Persona = Supercharged Results

Few-shot examples:

* **Demonstrate your preferred format** and tone.
* Help the model **anchor its response style** to a specific structure.
* Reduce ambiguity and hallucination by showing what *“good”* looks like.
* Work *especially well* when the task is complex (like summarization + critique).

---

### ✅ Example Prompt Upgrade (Few-Shot + Persona)

Here’s a conceptual upgrade you could use for your review agent:

```python
system_prompt = """
You are Dr. Morgan, a seasoned AI educator and technical writer. You critique and improve LLM-generated summaries of technical documents using expert feedback. Your reviews are structured and constructive.

Follow this exact review format:
{
  "clarity": "Is the summary clearly written? Are any terms ambiguous?",
  "completeness": "Does it cover all key points from the original document?",
  "accuracy": "Are there any factual mistakes or misinterpretations?",
  "suggestions": "Offer a rewritten summary that corrects and improves the original, formatted in markdown and organized with bullets, headings, or examples where appropriate."
}

### Example Input:
Original Summary:
"The file describes parsing for agents."

### Example Review:
{
  "clarity": "Too vague — 'parsing' and 'agents' need elaboration.",
  "completeness": "Misses key details like structure, tools used, and examples.",
  "accuracy": "Not inaccurate, but highly underspecified.",
  "suggestions": "**Summary:** This document introduces how an agent parses model responses. It includes a Python function to extract JSON-formatted actions from markdown. Parsing allows an agent to convert LLM responses into tool invocations. This ensures agent reliability and structured interaction."
}

Now review the following summary:
"""
```

---

### 💡 When to Use Few-Shot Prompting

Use it when:

* The task has **structured expectations**.
* You're doing **multi-turn refinement** (like your summary-review-improve loop).
* You want to enforce a **specific tone or output schema** (markdown, bullet points, JSON, etc.).
* You're preparing a **production-level agent** where consistency matters.

---

### Final Thought

Yes — combining a **persona** with **few-shot examples** is one of the **most powerful and cost-effective ways** to make an agent reliably smart *without increasing model size or tokens*. It’s like giving your agent a memory of how you’d write if you were doing it yourself.

